In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader

import sys
sys.path.append("..")
from src.models import Wav2Vec2_RawNet2
from src.datamodules import FixMatchDataset

In [2]:
# model_name_or_path = "facebook/wav2vec2-base"
# feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
# sampling_rate = feature_extractor.sampling_rate

path = "/home/joono/joono/joono/DV_DV.Deep/ad68hso6/checkpoints/best-checkpoint.ckpt"
# model = Wav2Vec2Facebook.load_from_checkpoint(path, args={})
model = Wav2Vec2_RawNet2.load_from_checkpoint(path, args={})

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


======== Trainable Lora layers ===========
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024])
torch.Size([1024])
torch.Size([4096, 1024])
torch.Size([4096])
torch.Size([1024, 4096])
torch.Size([1024])
torch.Size([1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024])
torch.Size([1024])
torch.Size([4096, 1024])
torch.Size([4096])
torch.Size([1024, 4096])
torch.Size([1024])
torch.Size([1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([1024])
torch.Size([1024])
torch.Size([4096, 1024])
torch.Size([4096])
torch.Size([1024, 4

In [3]:
# Collate 함수 정의
def collate_fn(batch):
    # signals = zip(*batch)
    signals = batch
    max_length = max([signal.size(0) for signal in signals])
    padded_signals = torch.zeros(len(signals), max_length)
    for i, signal in enumerate(signals):
        padded_signals[i, :signal.size(0)] = signal
    # labels = torch.tensor(labels)
    return padded_signals

In [4]:
test_df = pd.read_csv('../dataset/test.csv', index_col=None)
test_df['path'] = '../dataset/' + test_df['path'].str[1:]
# test_dataset = TestDataset(test_df)
# test_dataset = AASISTCenterLossDataset(test_df, train_mode=False)
test_dataset = FixMatchDataset(df=test_df, train_mode=False)
# test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32)
test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32, collate_fn=collate_fn)

/home/anaconda3/envs/joono/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
def inference(model, test_loader, device):
    model.to(device)
    model = model.eval()
    categories = []
    with torch.no_grad():
        # for inputs, labels in tqdm(test_loader):
        for inputs in tqdm(test_loader):
            
            inputs = inputs.to(device)
            
            category = model(inputs)

            categories += category.cpu().detach().numpy().tolist()
            
    return  categories

In [6]:
categories = inference(model=model, test_loader=test_loader, device='cuda:0')

100%|██████████| 1563/1563 [27:39<00:00,  1.06s/it]


In [7]:
categories_np = np.array(categories)
categories_class = np.argmax(categories_np, axis=1)
print(len(categories_class))

categories_softmax = np.exp(categories_np) / np.sum(np.exp(categories_np), axis=1, keepdims=True)
print(len(categories_softmax))

category_probs = np.zeros((len(categories_softmax), 2))
category_count = [0, 0, 0, 0]
for row in range(len(category_probs)):
    cat_max_idx = np.argmax(categories_softmax[row])
    category_count[cat_max_idx] += 1
    prob = categories_softmax[row, cat_max_idx]
    
    print(f"cat_max: {cat_max_idx}, prob: {prob}")
    
    if      cat_max_idx == 0: category_probs[row] = np.array([0.4, 0.4])
    elif    cat_max_idx == 1: category_probs[row] = np.array([0.6, 0.4])
    elif    cat_max_idx == 2: category_probs[row] = np.array([0.4, 0.6])
    elif    cat_max_idx == 3: category_probs[row] = np.array([0.6, 0.6])
category_count

50000
50000
cat_max: 3, prob: 0.8018300706667162
cat_max: 2, prob: 0.9108344333042332
cat_max: 2, prob: 0.8536654942012634
cat_max: 3, prob: 0.8401922703186122
cat_max: 2, prob: 0.926658456804596
cat_max: 2, prob: 0.9085840076225109
cat_max: 2, prob: 0.9180139099197706
cat_max: 2, prob: 0.7887499343116672
cat_max: 2, prob: 0.6736269068427365
cat_max: 3, prob: 0.869117467701515
cat_max: 2, prob: 0.92279077510236
cat_max: 2, prob: 0.9291531180671494
cat_max: 2, prob: 0.9155818234594113
cat_max: 3, prob: 0.8677269925305829
cat_max: 2, prob: 0.927170835156021
cat_max: 3, prob: 0.8632716599429424
cat_max: 2, prob: 0.9034780220316735
cat_max: 2, prob: 0.9107016647483783
cat_max: 2, prob: 0.9105830125902471
cat_max: 2, prob: 0.9074928179410097
cat_max: 2, prob: 0.9269150570817525
cat_max: 2, prob: 0.9222841186394771
cat_max: 2, prob: 0.9144761099502356
cat_max: 2, prob: 0.925529109061416
cat_max: 2, prob: 0.9209503083329441
cat_max: 2, prob: 0.9304961638135828
cat_max: 2, prob: 0.912835898194

[352, 2867, 40023, 6758]

In [8]:
submit = pd.read_csv('/home/joono/joono/dataset/sample_submission.csv')

fake, real = 0, 1

for i in tqdm(range(len(categories_np))):
    submit.iloc[i, 1:] = category_probs[i]

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_2113359/768754599.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submit.iloc[i, 1:] = category_probs[i]
/tmp/ipykernel_2113359/768754599.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submit.iloc[i, 1:] = category_probs[i]
100%|██████████| 50000/50000 [00:46<00:00, 1063.94it/s]


In [9]:
submit.to_csv('wav2vec2-rawnet2-centerloss-0406-2.csv', index=False, encoding='utf-8')

In [ ]:
import os
import pathlib
from glob import glob

In [ ]:

ids, paths = [], []
for ul_path in glob("/home/work/joono/joono/dataset/unlabeled_data/*"):
    basename = os.path.basename(ul_path)
    id = basename.split(".")[0]
    path = os.path.join(".", "unlabeled_data", basename)
    
    ids.append(id)
    paths.append(path)

In [ ]:
df = pd.DataFrame({
    "id": ids,
    "path": paths
})

In [ ]:
df.to_csv("unlabled_data.csv", index=False)

In [ ]:
paths[763]